In [1]:
import os

from sklearn.model_selection import train_test_split
import matplotlib
import argparse
import random
import numpy as np
import csv
import pandas as pd
from datascience import *
import tensorflow as tf
from sklearn.feature_extraction.text import CountVectorizer
from keras.models import Model
from keras.models import Sequential, load_model

from keras.layers import *
from keras.optimizers import Adam
from keras import Input
from keras.preprocessing.text import Tokenizer
from keras.utils import plot_model
import keras
#from keras.backend.tensorflow_backend import set_session
#config = tf.ConfigProto()
# config.gpu_options.allocator_type = 'BFC' #A "Best-fit with coalescing" algorithm, simplified from a version of dlmalloc.
# config.gpu_options.per_process_gpu_memory_fraction = 0.7
#config.gpu_options.allow_growth =True

#set_session(tf.Session(config=config)) 
import re
os.environ['PYTHONHASHSEED'] = '0'
random.seed(1)
np.random.seed(1)
tf.set_random_seed(1)
from keras.callbacks import *

Using TensorFlow backend.


In [2]:
def sanitize(string):
    words = string.split(' ')
    return words

In [3]:
#read csv

train = pd.read_csv("PAWS_QQP_train.tsv", sep='\t')
dev = pd.read_csv("PAWS_QQP_dev_and_test.tsv", sep='\t')
test = pd.read_csv("PAWS_QQP_dev_and_test.tsv", sep='\t')

print(train[:5])

train['title1_tokenized'] = \
    train.loc[:, 'sentence1'] \
         .apply(sanitize)
train['title2_tokenized'] = \
    train.loc[:, 'sentence2'] \
         .apply(sanitize)


dev['title1_tokenized'] = \
    dev.loc[:, 'sentence1'] \
         .apply(sanitize)
dev['title2_tokenized'] = \
    dev.loc[:, 'sentence2'] \
         .apply(sanitize)


test['title1_tokenized'] = \
    test.loc[:, 'sentence1'] \
         .apply(sanitize)
test['title2_tokenized'] = \
    test.loc[:, 'sentence2'] \
         .apply(sanitize)

   id                                          sentence1  \
0   1  b'Will a message still say blocked if you were...   
1   2  b'How can you treat ocd ? Is there any helpful...   
2   3  b'If you do not do anything how you are motiva...   
3   4  b'Why is new in system verily constructor not ...   
4   5  b'What are the most common traffic convictions...   

                                           sentence2  label  
0  b'Will a message still say delivered if you we...      0  
1  b'How can you treat OCD ? Is there any helpful...      1  
2  b'If you do not seek anything how you are moti...      0  
3  b'Why constructor new in system verilog is not...      0  
4  b'What are the most common traffic convictions...      0  


In [4]:
MAX_NUM_WORDS = 10000
tokenizer = keras \
    .preprocessing \
    .text \
    .Tokenizer(num_words=MAX_NUM_WORDS)

train_corpus_x1 = train.title1_tokenized
train_corpus_x2 = train.title2_tokenized
train_corpus = pd.concat([
    train_corpus_x1, train_corpus_x2])
#corpus.shape
tokenizer.fit_on_texts(train_corpus)
tokenizer.texts_to_sequences(train_corpus)
x1_train = tokenizer \
    .texts_to_sequences(train_corpus_x1)
x2_train = tokenizer \
    .texts_to_sequences(train_corpus_x2)


print(tokenizer.document_count)




dev_corpus_x1 = dev.title1_tokenized
dev_corpus_x2 = dev.title2_tokenized
dev_corpus = pd.concat([
    dev_corpus_x1, dev_corpus_x2])
#corpus.shape
tokenizer.fit_on_texts(dev_corpus)
tokenizer.texts_to_sequences(dev_corpus)
x1_dev = tokenizer \
    .texts_to_sequences(dev_corpus_x1)
x2_dev = tokenizer \
    .texts_to_sequences(dev_corpus_x2)


test_corpus_x1 = test.title1_tokenized
test_corpus_x2 = test.title2_tokenized
test_corpus = pd.concat([
    test_corpus_x1, test_corpus_x2])
#corpus.shape
tokenizer.fit_on_texts(test_corpus)
tokenizer.texts_to_sequences(test_corpus)
x1_test = tokenizer \
    .texts_to_sequences(test_corpus_x1)
x2_test = tokenizer \
    .texts_to_sequences(test_corpus_x2)

word_index = tokenizer.word_index


print(tokenizer.document_count)

23726
26402


In [5]:
max_seq_len1 = max([
    len(seq) for seq in x1_train])
print(max_seq_len1)

50


In [6]:


MAX_SEQUENCE_LENGTH = 50  #better to have words covered than uncovered
x1_train = keras \
    .preprocessing \
    .sequence \
    .pad_sequences(x1_train, 
                   maxlen=MAX_SEQUENCE_LENGTH)


x2_train = keras \
    .preprocessing \
    .sequence \
    .pad_sequences(x2_train, 
                   maxlen=MAX_SEQUENCE_LENGTH)

x1_dev = keras \
    .preprocessing \
    .sequence \
    .pad_sequences(x1_dev, 
                   maxlen=MAX_SEQUENCE_LENGTH)


x2_dev = keras \
    .preprocessing \
    .sequence \
    .pad_sequences(x2_dev, 
                   maxlen=MAX_SEQUENCE_LENGTH)


x1_test = keras \
    .preprocessing \
    .sequence \
    .pad_sequences(x1_test, 
                   maxlen=MAX_SEQUENCE_LENGTH)

x2_test = keras \
    .preprocessing \
    .sequence \
    .pad_sequences(x2_test, 
                   maxlen=MAX_SEQUENCE_LENGTH)


y_train = train.label

y_train = keras \
    .utils \
    .to_categorical(y_train, num_classes=2)


y_dev = dev.label

y_dev = keras \
    .utils \
    .to_categorical(y_dev, num_classes=2)



y_test = test.label

y_test = keras \
    .utils \
    .to_categorical(y_test, num_classes=2)

x1_test[:3]

array([[   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,   11,  153,    2,  265,  521,   10,
           2, 1441,  758,    8,    5,   30,   12,  236,  521,  112,    3,
           2, 4981,  758,    6, 4982,    1],
       [   0,    0,    0,    0,    0,    0,    0,    0,    0,    0, 1850,
         215,    9,  101,  689,  297, 4957,   71,    5,    9,  120,   47,
         108,   28,   54, 1034,   84,   71,   52,   50,    9,  166,  259,
         108,  718,   71,  108, 1037,   71,    5,  853,   71,   20,   59,
          12,    9,  120,   79,  103,   19],
       [   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,  122,   12,    9,   96,   26,
          93,  209,  146,   41,  690, 2899,   50,    9,   12,   54,   29,
         752,    3,   26,  185,  146,   34,   81,    7,    2,  690, 2899,
          57, 1030,   

In [7]:
#BiLSTM?
NUM_CLASSES=2 #boolean; 1 or 0

#rough estimate

MAX_SEQUENCE_LENGTH = 50 #how long one sentence is

#this is arbitrary?

NUM_LSTM_UNITS = 128 #output dimension


NUM_EMBEDDING_DIM = 256

top_input = Input(
    shape=(MAX_SEQUENCE_LENGTH, ),    #this is the first sentence
    dtype='int32')

bm_input = Input(
    shape=(MAX_SEQUENCE_LENGTH, ),   #this is the second
    dtype='int32')


embedding_layer = Embedding(
    MAX_NUM_WORDS, NUM_EMBEDDING_DIM)

top_embedded = embedding_layer(
    top_input)
bm_embedded = embedding_layer(
    bm_input)


shared_lstm = Bidirectional(LSTM(NUM_LSTM_UNITS))
top_output = shared_lstm(top_embedded)
bm_output = shared_lstm(bm_embedded)


merged = concatenate(
    [top_output, bm_output], 
    axis=-1)


dense =  Dense(
    units=NUM_CLASSES, 
    activation='sigmoid')
predictions = dense(merged)



model = Model(
    inputs=[top_input, bm_input], 
    outputs=predictions)


model.summary()

model.compile(
    optimizer='Adam',
    loss='categorical_crossentropy',
    metrics=['accuracy'])

BATCH_SIZE = 512

NUM_EPOCHS = 15

earlyStopping = EarlyStopping(monitor='val_loss', patience=10, verbose=0, mode='min')
mcp_save = ModelCheckpoint('BiLSTM_PAWS_QQP.h5', save_best_only=True, monitor='val_loss', mode='min')


Instructions for updating:
Colocations handled automatically by placer.
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 50)           0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            (None, 50)           0                                            
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, 50, 256)      2560000     input_1[0][0]                    
                                                                 input_2[0][0]                    
__________________________________________________________________________________________________
bidirectional_1 (Bidirectional) (None

In [9]:
!nvidia-smi


Fri Jun 21 00:36:16 2019       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 418.56       Driver Version: 418.56       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  GeForce GTX 1080    Off  | 00000000:02:00.0 Off |                  N/A |
|  0%   47C    P2    37W / 200W |    185MiB /  8118MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

In [19]:
history = model.fit(x=[x1_train, x2_train], y=y_train, batch_size=BATCH_SIZE, epochs=NUM_EPOCHS, callbacks=[earlyStopping, mcp_save], validation_data=([x1_dev, x2_dev], y_dev),
    
    shuffle=True
)

Train on 11863 samples, validate on 669 samples
Epoch 1/15
11863/11863 [==============================] - 11s 936us/step - loss: 0.6276 - acc: 0.6817 - val_loss: 0.5632 - val_acc: 0.7220
Epoch 2/15
11863/11863 [==============================] - 8s 646us/step - loss: 0.5310 - acc: 0.7381 - val_loss: 0.5440 - val_acc: 0.7489
Epoch 3/15
11863/11863 [==============================] - 8s 646us/step - loss: 0.4195 - acc: 0.8065 - val_loss: 0.6154 - val_acc: 0.7220
Epoch 4/15
11863/11863 [==============================] - 8s 647us/step - loss: 0.3324 - acc: 0.8576 - val_loss: 0.7137 - val_acc: 0.6816
Epoch 5/15
11863/11863 [==============================] - 8s 645us/step - loss: 0.2545 - acc: 0.8989 - val_loss: 0.8329 - val_acc: 0.6786
Epoch 6/15
11863/11863 [==============================] - 8s 647us/step - loss: 0.1906 - acc: 0.9244 - val_loss: 1.1302 - val_acc: 0.6218
Epoch 7/15
11863/11863 [==============================] - 8s 659us/step - loss: 0.1467 - acc: 0.9450 - val_loss: 1.1623 - v

In [11]:
model = load_model('BiLSTM_PAWS_QQP.h5')
predicts = model.predict(
    [x1_test, x2_test])

Instructions for updating:
Use tf.cast instead.


In [12]:
real1=test.sentence1
real2=test.sentence2
realCat=test.label
no = []
err1 = []
err2 = []
wrongCat = []


for x in range(669):
    if (np.argmax(predicts[x])!=(test.label[x])):
        no=np.append(no, x)
        err1=np.append(err1, real1[x])
        err2=np.append(err2, real2[x])
        wrongCat = np.append(wrongCat, realCat[x])
print(len(wrongCat))

194


In [16]:
error_analysis = Table()


error_analysis.with_column("sent1", err1, "sent2", err2,)

sent1,sent2
b'What are the driving rules in Georgia versus Mississip ...,b'What are the driving rules in Mississippi versus Georg ...
"b""Does AT & T have any specific plans for DirecTV 's ROO ...","b""Does DirecTV have any specific plans for AT & T 's ROO ..."
b'How do I start learning electronic music production ? ...,b'How do I begin learning electronic music production ? ...
b'What are the differences between abiotic factors and b ...,b'What are the differences between biotic factors and ab ...
"b'What is actual meaning of life ? Indeen , it depend on ...","b'What is other meaning of life ? Indeen , it depend on ..."
b'How safe are Indian Rs 500 and Rs 2000 new currency no ...,b'How safe is new Rs 500 and Rs 2000 Indian currency not ...
b'Why do some people write with their left hand and some ...,b'Why do some people write with their right hand and som ...
b'Do Chinese/Korean people think they look like Japanese ...,b'Do Japanese people think they look like Chinese/Korean ...
"b""`` Are `` if I were ... `` and `` if I was ... `` both ...","b""Are `` if I was '' and `` if I were '' both grammatica ..."
b'Where can I find French film or tv shows with french s ...,b'Where can I find french film or tv shows with French s ...
